In [2]:
!pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [25]:
import os
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp
from collections import Counter
from tqdm import trange
import pprint
from IPython.display import display, Math

# SymPy vetting

In [58]:
import sympy as sp

# 1) State variables: 4 genes + 12 dynamic thresholds
A, B, C, D = sp.symbols('A B C D')
KBA, KCA, KDA = sp.symbols('K_BA K_CA K_DA')
KAB, KCB, KDB = sp.symbols('K_AB K_CB K_DB')
KAC, KBC, KDC = sp.symbols('K_AC K_BC K_DC')
KAD, KBD, KCD = sp.symbols('K_AD K_BD K_CD')

# 2) Kinetic parameters
gA, gB, gC, gD = sp.symbols('Prod_of_A Prod_of_B Prod_of_C Prod_of_D')
dA, dB, dC, dD = sp.symbols('Deg_of_A Deg_of_B Deg_of_C Deg_of_D')

# shifted‐Hill λ & n for each edge
lamBA, nBA = sp.symbols('Inh_of_BToA Num_of_BToA')
lamCA, nCA = sp.symbols('Inh_of_CToA Num_of_CToA')
lamDA, nDA = sp.symbols('Inh_of_DToA Num_of_DToA')
lamAB, nAB = sp.symbols('Inh_of_AToB Num_of_AToB')
lamCB, nCB = sp.symbols('Inh_of_CToB Num_of_CToB')
lamDB, nDB = sp.symbols('Inh_of_DToB Num_of_DToB')
lamAC, nAC = sp.symbols('Inh_of_AToC Num_of_AToC')
lamBC, nBC = sp.symbols('Inh_of_BToC Num_of_BToC')
lamDC, nDC = sp.symbols('Inh_of_DToC Num_of_DToC')
lamAD, nAD = sp.symbols('Inh_of_AToD Num_of_AToD')
lamBD, nBD = sp.symbols('Inh_of_BToD Num_of_BToD')
lamCD, nCD = sp.symbols('Inh_of_CToD Num_of_CToD')

# 3) Epigenetic‐feedback parameters
a0_BA, alpha_BA = sp.symbols('a0_BA alpha_BA')
a0_CA, alpha_CA = sp.symbols('a0_CA alpha_CA')
a0_DA, alpha_DA = sp.symbols('a0_DA alpha_DA')
a0_AB, alpha_AB = sp.symbols('a0_AB alpha_AB')
a0_CB, alpha_CB = sp.symbols('a0_CB alpha_CB')
a0_DB, alpha_DB = sp.symbols('a0_DB alpha_DB')
a0_AC, alpha_AC = sp.symbols('a0_AC alpha_AC')
a0_BC, alpha_BC = sp.symbols('a0_BC alpha_BC')
a0_DC, alpha_DC = sp.symbols('a0_DC alpha_DC')
a0_AD, alpha_AD = sp.symbols('a0_AD alpha_AD')
a0_BD, alpha_BD = sp.symbols('a0_BD alpha_BD')
a0_CD, alpha_CD = sp.symbols('a0_CD alpha_CD')

# global timescale
beta = sp.symbols('beta')

# 4) Shifted‐Hill using dynamic K
def shifted_inh(H, lam, K, n):
    hill = 1/(1 + (H/K)**n)
    return (1 - lam)*hill + lam

# 5) Gene‐expression ODEs
exprA = (
    gA
    * shifted_inh(B, lamBA, KBA, nBA)
    * shifted_inh(C, lamCA, KCA, nCA)
    * shifted_inh(D, lamDA, KDA, nDA)
    - dA*A
)
exprB = (
    gB
    * shifted_inh(A, lamAB, KAB, nAB)
    * shifted_inh(C, lamCB, KCB, nCB)
    * shifted_inh(D, lamDB, KDB, nDB)
    - dB*B
)
exprC = (
    gC
    * shifted_inh(A, lamAC, KAC, nAC)
    * shifted_inh(B, lamBC, KBC, nBC)
    * shifted_inh(D, lamDC, KDC, nDC)
    - dC*C
)
exprD = (
    gD
    * shifted_inh(A, lamAD, KAD, nAD)
    * shifted_inh(B, lamBD, KBD, nBD)
    * shifted_inh(C, lamCD, KCD, nCD)
    - dD*D
)

# 6) Epigenetic‐feedback ODEs (continuous form)
dKBA = (a0_BA - KBA - alpha_BA*B) / beta
dKCA = (a0_CA - KCA - alpha_CA*C) / beta
dKDA = (a0_DA - KDA - alpha_DA*D) / beta

dKAB = (a0_AB - KAB - alpha_AB*A) / beta
dKCB = (a0_CB - KCB - alpha_CB*C) / beta
dKDB = (a0_DB - KDB - alpha_DB*D) / beta

dKAC = (a0_AC - KAC - alpha_AC*A) / beta
dKBC = (a0_BC - KBC - alpha_BC*B) / beta
dKDC = (a0_DC - KDC - alpha_DC*D) / beta

dKAD = (a0_AD - KAD - alpha_AD*A) / beta
dKBD = (a0_BD - KBD - alpha_BD*B) / beta
dKCD = (a0_CD - KCD - alpha_CD*C) / beta

# 7) Pack into one list of 16 ODEs
odes = [
    exprA, exprB, exprC, exprD,
    dKBA, dKCA, dKDA,
    dKAB, dKCB, dKDB,
    dKAC, dKBC, dKDC,
    dKAD, dKBD, dKCD
]

# 8) Parameter list in the exact same order
# params = [
#     gA, gB, gC, gD,
#     dA, dB, dC, dD,
#     lamBA, nBA, lamCA, nCA, lamDA, nDA,
#     lamAB, nAB, lamCB, nCB, lamDB, nDB,
#     lamAC, nAC, lamBC, nBC, lamDC, nDC,
#     lamAD, nAD, lamBD, nBD, lamCD, nCD,
#     a0_BA, alpha_BA, a0_CA, alpha_CA, a0_DA, alpha_DA,
#     a0_AB, alpha_AB, a0_CB, alpha_CB, a0_DB, alpha_DB,
#     a0_AC, alpha_AC, a0_BC, alpha_BC, a0_DC, alpha_DC,
#     a0_AD, alpha_AD, a0_BD, alpha_BD, a0_CD, alpha_CD,
#     beta
# ]

params = [
    gA, gB, gC, gD,                             # Prod_of_A to Prod_of_D
    dA, dB, dC, dD,                             # Deg_of_A to Deg_of_D

    a0_BA, nBA, lamBA,                          # B→A
    a0_CA, nCA, lamCA,                          # C→A
    a0_DA, nDA, lamDA,                          # D→A

    a0_AB, nAB, lamAB,                          # A→B
    a0_CB, nCB, lamCB,                          # C→B
    a0_DB, nDB, lamDB,                          # D→B

    a0_AC, nAC, lamAC,                          # A→C
    a0_BC, nBC, lamBC,                          # B→C
    a0_DC, nDC, lamDC,                          # D→C

    a0_AD, nAD, lamAD,                          # A→D
    a0_BD, nBD, lamBD,                          # B→D
    a0_CD, nCD, lamCD,                          # C→D

    alpha_BA, alpha_CA, alpha_DA,              # alpha_*
    alpha_AB, alpha_CB, alpha_DB,
    alpha_AC, alpha_BC, alpha_DC,
    alpha_AD, alpha_BD, alpha_CD,

    beta                                        # beta
]


In [59]:
odes_simpl = [sp.simplify(e) for e in odes]

for equation in odes_simpl:
    display(Math(sp.latex(equation)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Lambdifying 

In [67]:
import os
import pandas as pd

# ────────────────────────────────────────────────────────────────────────────────
# 0) Define all 12 alpha hyperparameters and beta here:
# ────────────────────────────────────────────────────────────────────────────────
ALPHA_BA = 0
ALPHA_CA = 0
ALPHA_DA = 0

ALPHA_AB = 0
ALPHA_CB = 0
ALPHA_DB = 0

ALPHA_AC = 0
ALPHA_BC = 0
ALPHA_DC = 0

ALPHA_AD = 0
ALPHA_BD = 0
ALPHA_CD = 0

BETA_VALUE = 100.0

# ────────────────────────────────────────────────────────────────────────────────
# 1) Load your TS.prs and TS_parameters.dat
# ────────────────────────────────────────────────────────────────────────────────
pt          = "/Users/hiteshkandarpa/Desktop/IISC/Summer'25/Code/initial_sims/Toggle_tetrahedron/Hypothesis_test"
names_file  = os.path.join(pt, "TS.prs")
params_file = os.path.join(pt, "TS_parameters.dat")

# read column names for columns 3+ from the .prs file
with open(names_file, 'r') as f:
    lines = [ln.strip() for ln in f if ln.strip()]
kinetic_names = [ln.split()[0] for ln in lines[1:]]

# read the full 10k–row table
df_all = pd.read_csv(
    params_file,
    sep=r'\s+',
    header=None,
    names=["S_no", "Reported_states"] + kinetic_names,
    
)

# ────────────────────────────────────────────────────────────────────────────────
# 2) Randomly sample selected number of parameter sets
# ────────────────────────────────────────────────────────────────────────────────
# pars = df_all#.iloc[:20] #.reset_index(drop=True)
pars = df_all.sample(n=100, random_state=42).reset_index(drop=True)

# ────────────────────────────────────────────────────────────────────────────────
# 3) Attach the 12 alpha’s and beta as explicit columns
# ────────────────────────────────────────────────────────────────────────────────
pars['alpha_BA'] = ALPHA_BA
pars['alpha_CA'] = ALPHA_CA
pars['alpha_DA'] = ALPHA_DA

pars['alpha_AB'] = ALPHA_AB
pars['alpha_CB'] = ALPHA_CB
pars['alpha_DB'] = ALPHA_DB

pars['alpha_AC'] = ALPHA_AC
pars['alpha_BC'] = ALPHA_BC
pars['alpha_DC'] = ALPHA_DC

pars['alpha_AD'] = ALPHA_AD
pars['alpha_BD'] = ALPHA_BD
pars['alpha_CD'] = ALPHA_CD

pars['beta']     = BETA_VALUE

# Now in your simulation loop you can do:
#
# for idx, row in pars.iterrows():
#     s_no   = int(row['S_no'])
#     # pull the epigenetic params:
#     alpha_BA = row['alpha_BA']
#     alpha_CA = row['alpha_CA']
#     # … etc …
#     beta     = row['beta']
#
#     # pull the RACIPE‐sampled params in your chosen order:
#     param_vals = [float(row[name]) for name in params_for_fnum]
#
#     # then call f_num including the dynamic threshold state variables
#     # and your param_vals, using those alpha_XY and beta where needed


In [68]:
print(pars.columns.tolist()[2:])
print(kinetic_names)
print(params_for_fnum)
print(pars.columns.tolist()[2:]==params_for_fnum)
print(params)

['Prod_of_A', 'Prod_of_B', 'Prod_of_C', 'Prod_of_D', 'Deg_of_A', 'Deg_of_B', 'Deg_of_C', 'Deg_of_D', 'Trd_of_BToA', 'Num_of_BToA', 'Inh_of_BToA', 'Trd_of_CToA', 'Num_of_CToA', 'Inh_of_CToA', 'Trd_of_DToA', 'Num_of_DToA', 'Inh_of_DToA', 'Trd_of_AToB', 'Num_of_AToB', 'Inh_of_AToB', 'Trd_of_CToB', 'Num_of_CToB', 'Inh_of_CToB', 'Trd_of_DToB', 'Num_of_DToB', 'Inh_of_DToB', 'Trd_of_AToC', 'Num_of_AToC', 'Inh_of_AToC', 'Trd_of_BToC', 'Num_of_BToC', 'Inh_of_BToC', 'Trd_of_DToC', 'Num_of_DToC', 'Inh_of_DToC', 'Trd_of_AToD', 'Num_of_AToD', 'Inh_of_AToD', 'Trd_of_BToD', 'Num_of_BToD', 'Inh_of_BToD', 'Trd_of_CToD', 'Num_of_CToD', 'Inh_of_CToD', 'alpha_BA', 'alpha_CA', 'alpha_DA', 'alpha_AB', 'alpha_CB', 'alpha_DB', 'alpha_AC', 'alpha_BC', 'alpha_DC', 'alpha_AD', 'alpha_BD', 'alpha_CD', 'beta']
['Prod_of_A', 'Prod_of_B', 'Prod_of_C', 'Prod_of_D', 'Deg_of_A', 'Deg_of_B', 'Deg_of_C', 'Deg_of_D', 'Trd_of_BToA', 'Num_of_BToA', 'Inh_of_BToA', 'Trd_of_CToA', 'Num_of_CToA', 'Inh_of_CToA', 'Trd_of_DToA', '

In [69]:
# 8) Simplify and lambdify
odes_simpl = [sp.simplify(e) for e in odes]
f_num = sp.lambdify(
    (A, B, C, D,
     KBA, KCA, KDA,
     KAB, KCB, KDB,
     KAC, KBC, KDC,
     KAD, KBD, KCD
    ) + tuple(params),
    odes_simpl,
    'numpy'
)

In [70]:
edges_list = [
    ('B','A'), ('C','A'), ('D','A'),   # thresholds on B→A, C→A, D→A
    ('A','B'), ('C','B'), ('D','B'),   #    A→B, C→B, D→B
    ('A','C'), ('B','C'), ('D','C'),   #    A→C, B→C, D→C
    ('A','D'), ('B','D'), ('C','D')    #    A→D, B→D, C→D
]


In [71]:
# production & degradation
params_for_fnum = pars.columns.tolist()[2:]

# Parellelised Vectorised RACIPE

In [72]:
import numpy as np
from joblib import Parallel, delayed

# 3) Updated integrator that reads by column‐name
def integrate_euler_epigenetic(row, n_ics=100, dt=0.1, n_steps=20000, min_thresh=0.00001):
    # 3a) pull numeric parameter values in correct order
    pvals = [float(row[name]) for name in params_for_fnum]
    # 3b) initial gene levels
    scale = np.array([row[f"Prod_of_{G}"]/row[f"Deg_of_{G}"] for G in ['A','B','C','D']], float)
    Xs = np.random.uniform(0,1,(n_ics,4)) * scale[None,:]
    # 3c) initial thresholds a0 in the same edges_list order
    a0_vals = np.array([row[f"Trd_of_{H}To{G}"] for H,G in edges_list], float)
    Ts = np.broadcast_to(a0_vals, (n_ics,12)).copy()

    # 4) Euler loop
    for _ in range(n_steps):
        # unpack genes
        A_vec, B_vec, C_vec, D_vec = Xs.T
        # pack thresholds for f_num call
        K_args = tuple(Ts[:,i] for i in range(12))
        # call lambdified ODE
        # split derivatives
        dA, dB, dC, dD, *dK = f_num(
            A_vec, B_vec, C_vec, D_vec,
            *K_args,
            *pvals
        )
        # update genes
        Xs += dt * np.vstack((dA,dB,dC,dD)).T
        Xs[Xs<0] = 0
        # update thresholds + clamp
        dK_arr = np.vstack(dK).T  # shape (n_ics, 12)
        Ts   += dt * dK_arr
        Ts    = np.maximum(Ts, min_thresh) #clamping

    return Xs

# 5) process one row
def process_param(row, tol=1.0, n_ics=100, dt=0.1, n_steps=20000):
    s_no = int(row["S_no"])
    finals = []
    Xs = integrate_euler_epigenetic(row, n_ics, dt, n_steps)
    for x in Xs:
        if not any(np.allclose(x,f,atol=tol) for f in finals):
            finals.append(x.copy())
    flat = [np.log2(v) if v>0 else -np.inf for st in finals for v in st]
    return [s_no, len(finals)] + flat

# 6) parallel map
results = Parallel(n_jobs=-1)(
    delayed(process_param)(pars.iloc[i])
    for i in range(len(pars))
)

# Assemble DataFrame
genes = ['A','B','C','D']
max_states = max((len(r) - 2)//4 for r in results)
cols = ["S_no","n_states"] + [
    f"{g}_ss{s+1}"
    for s in range(max_states)
    for g in genes
]
df_verify_parallel = pd.DataFrame(results, columns=cols)
df_verify_parallel



,S_no,n_states,A_ss1,B_ss1,C_ss1,D_ss1,A_ss2,B_ss2,C_ss2,D_ss2,...,C_ss26,D_ss26,A_ss27,B_ss27,C_ss27,D_ss27,A_ss28,B_ss28,C_ss28,D_ss28
0,6253,4,-2.823376,2.423911,-5.578884,0.420618,0.655022,-3.654235,-6.732510,4.315845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4685,1,-8.304383,-6.302471,4.612390,0.442311,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1732,2,4.958178,0.332900,-4.427263,-3.202687,3.051190,-3.440966,2.034309,-6.238294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4743,3,0.872038,-5.447354,-2.417181,1.147364,-3.736550,1.020303,-5.518684,1.359277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4522,2,1.274530,-3.798898,1.731379,-4.592432,-5.097275,-5.324268,1.690822,1.768252,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3788,2,3.128824,-5.479004,-2.388414,1.360347,4.563283,-4.719805,3.487337,-4.520017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,9190,4,1.072567,-7.198504,1.003992,-4.187192,2.917500,-6.745531,-3.383207,0.695764,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,7826,2,1.849302,-3.586051,1.714841,-4.187726,-4.755060,-4.184216,2.260798,1.183269,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,7540,3,-3.190252,0.236260,-8.498943,1.081635,1.241480,0.327913,-1.857084,-4.240155,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Writing this to output file

In [73]:
# Filter out parameter sets with >10 steady states and write RACIPE‐style output

# genes and DataFrame from above
genes = ['A','B','C','D']

# 1) Drop rows with more than 10 steady states
df_filtered = df_verify_parallel[df_verify_parallel['n_states'] <= 10].copy()

# 2) Write to file in RACIPE format
output_path = os.path.join(pt, "RACIPE_custom_ss_epigenetic_0.dat")
with open(output_path, 'w') as fout:
    for _, row in df_filtered.iterrows():
        s_no     = int(row['S_no'])
        n_states = int(row['n_states'])
        # collect only the actual steady‐state columns
        vals = []
        for s in range(n_states):
            for g in genes:
                vals.append(row[f"{g}_ss{s+1}"])
        # compose and write line
        line = [s_no, n_states] + vals
        fout.write("\t".join(f"{v:.6g}" for v in line) + "\n")

print(f"Wrote {len(df_filtered)} parameter sets to {output_path}")


Wrote 98 parameter sets to /Users/hiteshkandarpa/Desktop/IISC/Summer'25/Code/initial_sims/Toggle_tetrahedron/Hypothesis_test/RACIPE_custom_ss_epigenetic_0.dat


# Measuring differences Between RACIPE Output and Vectorised Sympy RACIPE 

In [76]:

# Path to RACIPE solutions file
raci_path = "/Users/hiteshkandarpa/Desktop/IISC/Summer'25/Code/initial_sims/Toggle_tetrahedron/Hypothesis_test/combined_solutions2.dat"

# 1) Read just the first two columns: parameter set number and reported number of states
df_raci = pd.read_csv(
    raci_path,
    sep=r'\s+',
    header=None,
    usecols=[0, 1],
    names=['S_no', 'R_states']
)

# 2) Filter to only those parameter sets you simulated (in df_verify)
df_raci_sub = df_raci[df_raci['S_no'].isin(df_verify['S_no'])]

# 3) Merge with your simulated results
df_comp = df_raci_sub.merge(
    df_verify[['S_no', 'n_states']],
    on='S_no',
    how='inner'
)

# 4) Compare
df_comp['match'] = df_comp['R_states'] == df_comp['n_states']
n_total   = len(df_comp)
n_match   = df_comp['match'].sum()
n_diff    = n_total - n_match
diff_sets = df_comp.loc[~df_comp['match'], 'S_no'].tolist()

# 5) Print summary
print(f"Compared {n_total} parameter sets (only those you simulated).")
print(f"{n_match} agree on steady‐state count.")
print(f"{n_diff} differ: {diff_sets}")

# 6) Show comparison details
df_comp


Compared 44 parameter sets (only those you simulated).
23 agree on steady‐state count.
21 differ: [4, 8, 10, 11, 12, 14, 20, 22, 29, 31, 33, 34, 38, 39, 42, 43, 45, 46, 47, 49, 50]


,S_no,R_states,n_states,match
0,2,1,1,True
1,4,3,2,False
2,5,3,3,True
3,8,17,2,False
4,9,2,2,True
5,10,3,1,False
6,11,3,2,False
7,12,4,2,False
8,13,2,2,True
9,14,4,2,False
